In [23]:
import geopandas as gpd
import pandas as pd
import numpy as np

## Configuraciones Iniciales
ocupacion_distrito_2011_csv_path = r"C:\Users\Usuario\OneDrive\Escritorio\UOC\TFM\PEC3  - Implementacion\WorkStation\Datos\Variables\03 - Ocupacion\Distritos\Ocupados de 16 y más años_Ocupacion_Distritos_Censo_2011.csv"
ocupacion_distrito_2021_csv_path = r"C:\Users\Usuario\OneDrive\Escritorio\UOC\TFM\PEC3  - Implementacion\WorkStation\Datos\Variables\03 - Ocupacion\Distritos\Ocupados de 16 y más años_Ocupacion_Distritos_Censo_2021.csv"
cartografia_distrito_shp_path = r"C:\Users\Usuario\OneDrive\Escritorio\UOC\TFM\PEC3  - Implementacion\WorkStation\Datos\Shapefiles\da_cartografiaDistritoMunicipal-25830\da_cartografiaDistritoMunicipalPolygon.shp"

## Lectura inicial de datos ocupacion
df_ocupacion_distrito_2011 =  pd.read_csv(ocupacion_distrito_2011_csv_path,sep=";")
df_ocupacion_distrito_2021 =  pd.read_csv(ocupacion_distrito_2021_csv_path,sep=";")
df_cartografia_distrito = gpd.read_file(cartografia_distrito_shp_path)

# Ver las columnas
#print(df_ocupacion_distrito_2011.columns)
#print(df_ocupacion_distrito_2021.columns)
#print(df_cartografia_distrito.columns)

# Inspeccionar los primeros registros
#print(df_ocupacion_distrito_2011.head())
#print(df_ocupacion_distrito_2021.head())
#print(df_cartografia_distrito.head())

#Se realiza una normalizacion de los csv de ocupacion en dos columnas ocupacion y personas 
df_ocupacion_distrito_2011_long = pd.melt(df_ocupacion_distrito_2011, id_vars=["Municipio de residencia","Distrito de residencia"], var_name='Ocupacion', value_name='Personas')
#print(df_ocupacion_distrito_2011_long.columns)
#print(df_ocupacion_distrito_2011_long.head())
df_ocupacion_distrito_2021_long = pd.melt(df_ocupacion_distrito_2021, id_vars=["Municipio de residencia","Distrito "], var_name='Ocupacion', value_name='Personas')
#print(df_ocupacion_distrito_2021_long.columns)
#print(df_ocupacion_distrito_2021_long.head())

#Cambio el nombre de la columna 'Distrito ' a Distrito de residencia
df_ocupacion_distrito_2021_long.rename(columns={'Distrito ': 'Distrito de residencia'}, inplace=True)
#print(df_ocupacion_distrito_2021_long.columns)

#Contamos el numero de valores que hay en estos casos
num_filas_en_blanco_2011 = (df_ocupacion_distrito_2011_long['Personas'].isna()).sum()
num_filas_con_asterisco_2011 = (df_ocupacion_distrito_2011_long['Personas'] == "*").sum()
#print(num_filas_en_blanco_2011)#6
#print(num_filas_con_asterisco_2011)#10

## Eliminar filas ##
#Eliminar filas de municipio Total
df_ocupacion_distrito_2011_long = df_ocupacion_distrito_2011_long[df_ocupacion_distrito_2011_long['Municipio de residencia'] != 'Total']
df_ocupacion_distrito_2021_long = df_ocupacion_distrito_2021_long[df_ocupacion_distrito_2021_long['Municipio de residencia'] != 'Total']
#Eliminamos las filas de ocupaciones militares, no consta y totales y tambien distrito de residencia Total
df_ocupacion_distrito_2011_long = df_ocupacion_distrito_2011_long[df_ocupacion_distrito_2011_long['Ocupacion'] != '0 - Ocupaciones militares']
df_ocupacion_distrito_2021_long = df_ocupacion_distrito_2021_long[df_ocupacion_distrito_2021_long['Ocupacion'] != '0 - Ocupaciones militares']
df_ocupacion_distrito_2011_long = df_ocupacion_distrito_2011_long[df_ocupacion_distrito_2011_long['Ocupacion'] != 'No consta']
df_ocupacion_distrito_2021_long = df_ocupacion_distrito_2021_long[df_ocupacion_distrito_2021_long['Ocupacion'] != 'No consta']
df_ocupacion_distrito_2011_long = df_ocupacion_distrito_2011_long[df_ocupacion_distrito_2011_long['Ocupacion'] != 'Total']
df_ocupacion_distrito_2021_long = df_ocupacion_distrito_2021_long[df_ocupacion_distrito_2021_long['Ocupacion'] != 'Total']
df_ocupacion_distrito_2011_long = df_ocupacion_distrito_2011_long[df_ocupacion_distrito_2011_long['Distrito de residencia'] != 'Total']
df_ocupacion_distrito_2021_long = df_ocupacion_distrito_2021_long[df_ocupacion_distrito_2021_long['Distrito de residencia'] != 'Total']
#print(df_ocupacion_distrito_2011_long.head())
#print(df_ocupacion_distrito_2021_long.head())

## Añadir columnas ##
#Añadir columna NUMERO con el numero de distrito
df_ocupacion_distrito_2011_long["NUMERO"] = df_ocupacion_distrito_2011_long["Distrito de residencia"].str[-2:].astype(float)
df_ocupacion_distrito_2021_long["NUMERO"] = df_ocupacion_distrito_2021_long["Distrito de residencia"].str[-2:].astype(float)
#print(df_ocupacion_distrito_2011_long.columns)
#print(df_ocupacion_distrito_2021_long.columns)
#Añadir columna Id_Ocupacion
#Se puede hacer asi porque solo va de 1 a 9
df_ocupacion_distrito_2011_long["Id_Ocupacion"] = df_ocupacion_distrito_2011_long["Ocupacion"].str[0].astype(float)
df_ocupacion_distrito_2021_long["Id_Ocupacion"] = df_ocupacion_distrito_2021_long["Ocupacion"].str[0].astype(float)

## Gestion de Nulos y valores extraños ##
#Contamos el numero de valores que hay en estos casos despues de los ajustes anteriores (Eliminar filas)
num_filas_en_blanco_2011 = (df_ocupacion_distrito_2011_long['Personas'].isna()).sum()
num_filas_con_asterisco_2011 = (df_ocupacion_distrito_2011_long['Personas'] == "*").sum()
#print(num_filas_en_blanco_2011)#0
#print(num_filas_con_asterisco_2011)#10
#Sustituimos los valores * por NA
df_ocupacion_distrito_2011_long['Personas'] = df_ocupacion_distrito_2011_long['Personas'].replace('*', np.nan)
df_ocupacion_distrito_2021_long['Personas'] = df_ocupacion_distrito_2021_long['Personas'].replace('*', np.nan)
#Columna Personas como float
df_ocupacion_distrito_2011_long['Personas'] = df_ocupacion_distrito_2011_long['Personas'].astype(float)
df_ocupacion_distrito_2021_long['Personas'] = df_ocupacion_distrito_2021_long['Personas'].astype(float)
#Contamos el numero de valores tras los nuevos ajustes
num_filas_en_blanco_2021 = (df_ocupacion_distrito_2021_long['Personas'].isna()).sum()
num_filas_con_asterisco_2021 = (df_ocupacion_distrito_2021_long['Personas'] == "*").sum()
#print(num_filas_en_blanco_2021)#10
#print(num_filas_con_asterisco_2021)#0
# Calcular la desviación estándar antes de la imputación
desviacion_imputada_2011= df_ocupacion_distrito_2011_long.groupby('Distrito de residencia')['Personas'].std()
#print("Desviación estándar por distrito antes de imputar 3:", desviacion_imputada_2011)

## Calcular % Personas ocupacion por distrito con valores NAN ##
#Calcular el total de personas por distrito
df_ocupacion_distrito_2011_long['Total_personas_distrito'] = df_ocupacion_distrito_2011_long.groupby('Distrito de residencia')['Personas'].transform('sum')
df_ocupacion_distrito_2021_long['Total_personas_distrito'] = df_ocupacion_distrito_2021_long.groupby('Distrito de residencia')['Personas'].transform('sum')
#Calcular porcentaje sobre el total
df_ocupacion_distrito_2011_long['Personas %'] = (df_ocupacion_distrito_2011_long['Personas'] / df_ocupacion_distrito_2011_long['Total_personas_distrito'])
df_ocupacion_distrito_2021_long['Personas %'] = (df_ocupacion_distrito_2021_long['Personas'] / df_ocupacion_distrito_2021_long['Total_personas_distrito'])
#print(df_ocupacion_distrito_2011_long[df_ocupacion_distrito_2011_long['NUMERO'] == 1])
#print(df_ocupacion_distrito_2011_long[df_ocupacion_distrito_2011_long['NUMERO'] == 3])
#print(df_ocupacion_distrito_2011_long[df_ocupacion_distrito_2011_long['NUMERO'] == 5])
#print(df_ocupacion_distrito_2021_long[df_ocupacion_distrito_2021_long['NUMERO'] == 1])

## Calcular % Personas ocupacion por distrito con valor medio 3 ##
#Los valores con * tienen un valor entre 1-5 según el INE. Utilizaremos la media (3) como estimacion
#No obstante vamos a ver si la desviacion del dato cambia mucho respecto a lo anterior usando el 3
df_ocupacion_distrito_2011_long['Personas'] = df_ocupacion_distrito_2011_long['Personas'].replace(np.nan, 3)
df_ocupacion_distrito_2021_long['Personas'] = df_ocupacion_distrito_2021_long['Personas'].replace(np.nan, 3)
#Contamos el numero de valores que hay en estos casos despues de los ajustes anteriores otra vez
num_filas_en_blanco_2011 = (df_ocupacion_distrito_2011_long['Personas'].isna()).sum()
num_filas_con_asterisco_2011 = (df_ocupacion_distrito_2011_long['Personas'] == "*").sum()
#print(num_filas_en_blanco_2011)#0
#print(num_filas_con_asterisco_2011)#0
# Calcular la desviación estándar despues de la imputación 
desviacion_imputada_2011= df_ocupacion_distrito_2011_long.groupby('Distrito de residencia')['Personas'].std()
#print("Desviación estándar por distrito despues de imputar 3:", desviacion_imputada_2011)

#Calcular el total de personas por distrito
df_ocupacion_distrito_2011_long['Total_personas_distrito'] = df_ocupacion_distrito_2011_long.groupby('Distrito de residencia')['Personas'].transform('sum')
df_ocupacion_distrito_2021_long['Total_personas_distrito'] = df_ocupacion_distrito_2021_long.groupby('Distrito de residencia')['Personas'].transform('sum')
#Calcular porcentaje sobre el total
df_ocupacion_distrito_2011_long['Personas %'] = (df_ocupacion_distrito_2011_long['Personas'] / df_ocupacion_distrito_2011_long['Total_personas_distrito'])
df_ocupacion_distrito_2021_long['Personas %'] = (df_ocupacion_distrito_2021_long['Personas'] / df_ocupacion_distrito_2021_long['Total_personas_distrito'])
#print(df_ocupacion_distrito_2011_long[df_ocupacion_distrito_2011_long['NUMERO'] == 1])
#print(df_ocupacion_distrito_2011_long[df_ocupacion_distrito_2011_long['NUMERO'] == 3])
#print(df_ocupacion_distrito_2011_long[df_ocupacion_distrito_2011_long['NUMERO'] == 5])
#print(df_ocupacion_distrito_2021_long[df_ocupacion_distrito_2021_long['NUMERO'] == 1])

#Incluir campos de geolocalizacion
#2011
df_ocupacion_distrito_2011_long_mapped = pd.merge(df_ocupacion_distrito_2011_long, df_cartografia_distrito, on='NUMERO')
print(df_ocupacion_distrito_2011_long_mapped )
#2021
df_ocupacion_distrito_2021_long_mapped = pd.merge(df_ocupacion_distrito_2021_long, df_cartografia_distrito, on='NUMERO')
print(df_ocupacion_distrito_2021_long_mapped)


# Export
columnas_exportacion= ['NOMBRE', 'Ocupacion', 'Id_Ocupacion' ,'Personas %', 'ID_DISMUNI', 'NUMERO', 'geometry']
df_ocupacion_distrito_2011_long_mapped_export = df_ocupacion_distrito_2011_long_mapped[columnas_exportacion]
df_ocupacion_distrito_2021_long_mapped_export = df_ocupacion_distrito_2021_long_mapped[columnas_exportacion]

#df_ocupacion_distrito_2011_long_mapped_export.to_csv(r"C:\Users\Usuario\OneDrive\Escritorio\UOC\TFM\PEC3  - Implementacion\WorkStation\Datos\Variables\03 - Ocupacion\Distritos\2011_Distritos_Ocupacion.csv",index=False,sep=";")
#df_ocupacion_distrito_2021_long_mapped_export.to_csv(r"C:\Users\Usuario\OneDrive\Escritorio\UOC\TFM\PEC3  - Implementacion\WorkStation\Datos\Variables\03 - Ocupacion\Distritos\2021_Distritos_Ocupacion.csv",index=False,sep=";")

#Export 2 trabajos 'top'
#Los trabajos top son Directores y gerentes; Técnicos y profesionales científicos e intelectuales; Técnicos; profesionales de apoyo
df_ocupacion_distrito_2011_long_mapped_only_knoledge = df_ocupacion_distrito_2011_long_mapped[df_ocupacion_distrito_2011_long_mapped['Id_Ocupacion'].isin([1, 2, 3])]
df_ocupacion_distrito_2021_long_mapped_only_knoledge = df_ocupacion_distrito_2021_long_mapped[df_ocupacion_distrito_2021_long_mapped['Id_Ocupacion'].isin([1, 2, 3])]

# Agrupar por distrito y sumar 'Personas' y 'Porcentaje'
df_agrupado_2011 = df_ocupacion_distrito_2011_long_mapped_only_knoledge.groupby(['NOMBRE','Distrito de residencia','NUMERO'], as_index=False).agg({
    'Personas %': 'sum'
})

df_agrupado_2021 = df_ocupacion_distrito_2021_long_mapped_only_knoledge.groupby(['NOMBRE','Distrito de residencia','NUMERO'], as_index=False).agg({
    'Personas %': 'sum'
})

#Renombrar columna
df_agrupado_2011.rename(columns={'Personas %': 'kw_pct'}, inplace=True)
df_agrupado_2021.rename(columns={'Personas %': 'kw_pct'}, inplace=True)

print(df_agrupado_2011)
print(df_agrupado_2021)

df_agrupado_2011.to_csv(r"C:\Users\Usuario\OneDrive\Escritorio\UOC\TFM\PEC3  - Implementacion\WorkStation\Datos\Variables\03 - Ocupacion\Distritos\2011_Distritos_Ocupacion_V2.csv",index=False,sep=";")
df_agrupado_2021.to_csv(r"C:\Users\Usuario\OneDrive\Escritorio\UOC\TFM\PEC3  - Implementacion\WorkStation\Datos\Variables\03 - Ocupacion\Distritos\2021_Distritos_Ocupacion_V2.csv",index=False,sep=";")

   Municipio de residencia Distrito de residencia  \
0             29067 Málaga       Distrito 2906702   
1             29067 Málaga       Distrito 2906704   
2             29067 Málaga       Distrito 2906706   
3             29067 Málaga       Distrito 2906707   
4             29067 Málaga       Distrito 2906708   
..                     ...                    ...   
94            29067 Málaga       Distrito 2906710   
95            29067 Málaga       Distrito 2906711   
96            29067 Málaga       Distrito 2906701   
97            29067 Málaga       Distrito 2906703   
98            29067 Málaga       Distrito 2906705   

                      Ocupacion  Personas  NUMERO  Id_Ocupacion  \
0     1 - Directores y gerentes    2185.0     2.0           1.0   
1     1 - Directores y gerentes     220.0     4.0           1.0   
2     1 - Directores y gerentes     560.0     6.0           1.0   
3     1 - Directores y gerentes    1055.0     7.0           1.0   
4     1 - Directores y geren